In [1]:
import sys, io
import copy
import math
import numpy as np
import pandas as pd
from scipy.optimize import nnls
from fractions import Fraction
from sympy import factorint
from itertools import product, permutations, combinations
from typing import List, Optional, Tuple
import random



$$
{\Large
d = \sum_{r=0}^{m-1} 3^{m-1-r} \cdot 2^{s_r}
}
$$

That list of exponents ((s_0,\ldots,s_{m-1})) is what we call `s_list`.

---

### 🔹 Example 1: Representing (d=1)

We can take (m=1) and (s_0=0).
Then

$$
d = 3^{0} \cdot 2^{0} = 1.
$$

So `s_list = [0]` represents 1.

---

### 🔹 Example 2: Representing (d=2)

Again (m=1), (s_0=1):

$$
d = 3^{0} \cdot 2^{1} = 2.
$$

So `s_list = [1]` represents 2.

---

### 🔹 Example 3: Representing (d=3)

(3 = 3^{1}\cdot 2^{0} + 3^{0}\cdot 2^{0}).
So (m=2), (s_0 = 0, s_1 = 0):

$$
d = 3^{1}!·2^{0} + 3^{0}!·2^{0} = 3 + 1 = 4\text{(too large)}.
$$

We need (3), not 4, so that’s not correct.
Try (s_0=0, s_1=-1) (which would correspond to dividing by 2 once, not legal in our integer index system).
So for (d=3) you need to use a larger (m) and a mix of exponents, e.g.

$$
3 = 3^{1}!·2^{0} + 3^{0}!·2^{-1},
$$

but since we only allow integer, nonnegative exponents, we would instead increase (T) later to absorb the fractional (2^{-1}).

The key takeaway: not every small integer has a simple one-term representation, but by allowing a shift (T) later (multiplying the whole thing by (2^T)) you can make it work with non-negative exponents. That’s precisely why the lemma introduces the (T) shift.



In [2]:
def s_Sum(m, S):
    m = len(S)
    total = 0
    for r in range(m):
        total += ( (3**(m-1-r)) * (2**(S[r])) )
    return total
#

In [3]:
s_Sum(0, [0])

1

In [4]:
def generate_ordered_index_lists(m, N):    
    indexes = list(range(1, N))
    return [list(c) for c in combinations(indexes, m)]
#

In [5]:
generate_ordered_index_lists(2, 4)


[[1, 2], [1, 3], [2, 3]]

In [6]:
def prepend_0(L):
    LL = [0]
    LL.extend(L)
    return LL
#
prepend_0([1])

[0, 1]

In [7]:
def generate_S(m, N):
    if m == 0:
        yield [0]
    else:
        for L in map(prepend_0, generate_ordered_index_lists(m, N)):
            yield L
#

In [8]:
list(generate_S(2, 8))

[[0, 1, 2],
 [0, 1, 3],
 [0, 1, 4],
 [0, 1, 5],
 [0, 1, 6],
 [0, 1, 7],
 [0, 2, 3],
 [0, 2, 4],
 [0, 2, 5],
 [0, 2, 6],
 [0, 2, 7],
 [0, 3, 4],
 [0, 3, 5],
 [0, 3, 6],
 [0, 3, 7],
 [0, 4, 5],
 [0, 4, 6],
 [0, 4, 7],
 [0, 5, 6],
 [0, 5, 7],
 [0, 6, 7]]

In [9]:
def generate_S_vals(N):
    for i in range(N):
        for S in generate_S(i, N):
            val = s_Sum(i, S)
            yield (i, S, val, val % 4)

In [10]:
for tup in generate_S_vals(8):
    print(tup)

(0, [0], 1, 1)
(1, [0, 1], 5, 1)
(1, [0, 2], 7, 3)
(1, [0, 3], 11, 3)
(1, [0, 4], 19, 3)
(1, [0, 5], 35, 3)
(1, [0, 6], 67, 3)
(1, [0, 7], 131, 3)
(2, [0, 1, 2], 19, 3)
(2, [0, 1, 3], 23, 3)
(2, [0, 1, 4], 31, 3)
(2, [0, 1, 5], 47, 3)
(2, [0, 1, 6], 79, 3)
(2, [0, 1, 7], 143, 3)
(2, [0, 2, 3], 29, 1)
(2, [0, 2, 4], 37, 1)
(2, [0, 2, 5], 53, 1)
(2, [0, 2, 6], 85, 1)
(2, [0, 2, 7], 149, 1)
(2, [0, 3, 4], 49, 1)
(2, [0, 3, 5], 65, 1)
(2, [0, 3, 6], 97, 1)
(2, [0, 3, 7], 161, 1)
(2, [0, 4, 5], 89, 1)
(2, [0, 4, 6], 121, 1)
(2, [0, 4, 7], 185, 1)
(2, [0, 5, 6], 169, 1)
(2, [0, 5, 7], 233, 1)
(2, [0, 6, 7], 329, 1)
(3, [0, 1, 2, 3], 65, 1)
(3, [0, 1, 2, 4], 73, 1)
(3, [0, 1, 2, 5], 89, 1)
(3, [0, 1, 2, 6], 121, 1)
(3, [0, 1, 2, 7], 185, 1)
(3, [0, 1, 3, 4], 85, 1)
(3, [0, 1, 3, 5], 101, 1)
(3, [0, 1, 3, 6], 133, 1)
(3, [0, 1, 3, 7], 197, 1)
(3, [0, 1, 4, 5], 125, 1)
(3, [0, 1, 4, 6], 157, 1)
(3, [0, 1, 4, 7], 221, 1)
(3, [0, 1, 5, 6], 205, 1)
(3, [0, 1, 5, 7], 269, 1)
(3, [0, 1, 6, 7], 365, 

# S - Value discussion

The S values restrict our easy choices for D, but we only need a few numbers $mod_4(N) == 1 \text{ or } 3$ to succeed.

We desire an even and an odd length for each:

## $mod_4(N) == 1$

| S | value |
| --- | --- |
| \[0\] | 1 |
| \[0, 1\] | 5 |

## $mod_4(N) == 3$

| S | value |
| --- | --- |
| \[0, 2\] | 7 |
| \[0, 1, 2\] | 19 |


In [11]:
for i in range(25, 25+16, 2):
    found = False
    for d in [1,5,7,19]:
        if ((i - d) % 4) == 0:
            found = True
            print((i, d, (i-d)//4))
    if not found:
        print("%d not found"%(i))

(25, 1, 6)
(25, 5, 5)
(27, 7, 5)
(27, 19, 2)
(29, 1, 7)
(29, 5, 6)
(31, 7, 6)
(31, 19, 3)
(33, 1, 8)
(33, 5, 7)
(35, 7, 7)
(35, 19, 4)
(37, 1, 9)
(37, 5, 8)
(39, 7, 8)
(39, 19, 5)


In [39]:
def CollatzChain(collatzNumber):
    chain = [collatzNumber]
    while collatzNumber != 1:
        if collatzNumber & 1 == 0:
            collatzNumber = collatzNumber // 2
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
        chain.append(collatzNumber)
    return chain
#

def ChainPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#
def fractionFromNodeTup(tup):
    p2, p3, c = tup
    fract = Fraction(2**p2 - c, 3**p3)
    return (fract.numerator, fract.denominator)
#

def TupChainFromPath(chain_path):
    tup_chain = [(0, 0, 0)]
    for chain_item in chain_path:
        p2, p3, c = tup_chain[-1]
        if chain_item == "1":
            tup_chain. append((p2 + 1, p3, c))
        else:
            tup_chain. append((p2 + 1, p3 + 1, c*3 + 2**p2))
            
        fract = fractionFromNodeTup(tup_chain[-1])
    return tup_chain
#
def FractionFromPath(chain_path):
    tup = (0, 0, 0)
    for chain_item in chain_path:
        p2, p3, c = tup
        if chain_item == "1":
            tup = (p2 + 1, p3, c)
        else:
            tup = (p2 + 1, p3 + 1, c*3 + 2**p2)
        
    fract = fractionFromNodeTup(tup)
    return fract
#    
def downUpTup(tup):
    p2, p3, c = tup
    p2_01, p3_01, c_01 = (p2 + 2, p3 + 1, c*3 + 2**p2)
    return (p2_01, p3_01, c_01)
#
def TupChain(collatzNumber):
    chain_path = ChainPath(collatzNumber)
    return TupChainFromPath(chain_path)
    return tup_chain
#
def generationTups(n):
    G = [[(0, 0, 0)]]
    for i in range(1, n+1, 1):
        G.append([])
        tups = G[i-1]
        for tup in tups:
            p2, p3, c = tup
            G[i].extend([(p2 + 1, p3, c), (p2 + 1, p3 + 1, c*3 + 2**p2)])
    return G
#
def pathToTup(path):
    a = len(path)
    zeros = [i for i, b in enumerate(path) if b == '0']
    b = len(zeros)
    # compute c = sum_{j=1}^{k} 3^{k-j} * 2^{i_j - 1}
    c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in enumerate(zeros))
    fract = Fraction(2**a - c, 3**b)
    return path, (a, b, c), (fract.numerator, fract.denominator)
#
def reverseColllatz(collatz_num, generations):
    chains = [[collatz_num]]
    for i in range(generations):
        for chain in chains.copy():
            if ((chain[-1] -1) % 3) == 0:
                reverse_down_val = (chain[-1] - 1)//3
                if (reverse_down_val & 1) == 1:
                    # down only applies to odds
                    chain2 = chain.copy()
                    chain2.append(reverse_down_val)
                    chains.append(chain2)
            chain = chain.append(2*chain[-1])
    #
    return chains
#


def collatzNext(x):
    if x %2 == 0:
        return x//2
    return 3*x + 1
#
def fourPlusOneEdges(maxN):
    for n in range(maxN):
        start = 4*n + 1
        y = collatzNext(start)
        while y % 4 != 1:
            y = collatzNext(y)
        print((n, (y-1)//4))
#
def ToNPlusOneEdges(g, maxN):
    L = []
    multiplier = 2**g
    for n in range(maxN):
        start = multiplier*n + 1
        y = collatzNext(start)
        while y % multiplier != 1:
            y = collatzNext(y)
        L.append((n, (y-1)//multiplier))
    return L
#


def two_factors(n):
    """
    Returns the exponent of the largest power of 2 that divides n.
    For example, if n = 48 (binary 110000), the result is 4 because 2^4 = 16 divides 48.
    """
    if n <= 1:
        return 0 

    count = 0
    while (n & 1) == 0:
        n >>= 1
        count += 1
    return count
#
def gen_generation(a):
    seqs = product('10', repeat=a)
    for bits in seqs:
        label = ''.join(bits)
        zeros = [i for i, b in enumerate(bits) if b == '0']
        b = len(zeros)
        # compute c = sum_{j=0}^{k} 3^{k-j} * 2^{i_j - 1}
        c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in enumerate(zeros))
        f = Fraction(2**a - c, 3**b)
        yield (label, (a,b,c), (f.numerator, f.denominator))
#

In [40]:
list(gen_generation(2))

[('11', (2, 0, 0), (4, 1)),
 ('10', (2, 1, 2), (2, 3)),
 ('01', (2, 1, 1), (1, 1)),
 ('00', (2, 2, 5), (-1, 9))]

In [13]:
def compute_c_from_zeros(zeros: List[int]) -> int:
    if not zeros:
        return 0
    b = len(zeros)
    return sum((3 ** (b - j - 1)) * (1 << zeros[j]) for j in range(b))

def value_from_zeros(a: int, zeros: List[int]) -> Optional[int]:
    B = len(zeros)
    c = compute_c_from_zeros(zeros)
    num = (1 << a) - c
    denom = 3 ** B
    if num >= 0 and num % denom == 0:
        return num // denom
    return None


In [14]:
def canonical_T_insertion(a: int, zeros: List[int], d: int,
                          T_start: Optional[int] = None, T_max: int = 2000,
                          m_max: int = 6, s_max: int = 20
                          ) -> Optional[Tuple[int, List[int], int]]:
    """
    Inputs:
      a, zeros: original bitstring length and zero indices list (strictly increasing)
      d: digit (one of allowed values)
    Returns:
      (a_new, final_zeros, c_new) where final_zeros is the downshifted zero list
      that yields the value 4*n + d (with n calculated from (a,zeros)).
      Returns None if no T <= T_max and no s_list found in bounds.
    """
    b = len(zeros)
    # verify input consistency
    c_old = compute_c_from_zeros(zeros)
    if ((1 << a) - c_old) < 0 or ((1 << a) - c_old) % (3 ** b) != 0:
        raise ValueError("Input (a,zeros) inconsistent.")
    n = ((1 << a) - c_old) // (3 ** b)
    target = 4 * n + d

    # find canonical s_list for d
    s_list = find_s_list_for_d(d, m_max=m_max, s_max=s_max)
    if s_list is None:
        return None
    m = len(s_list)

    last_old = zeros[-1] if zeros else -1
    minT = max(a + 2, last_old + 2) + 1
    if T_start is None:
        T = minT
    else:
        T = max(T_start, minT)
    a_new = a + 2

    while T <= T_max:
        new_ms = [T + s for s in s_list]
        tilde = [z + 2 + T for z in zeros]
        zeros_combined = sorted(new_ms + tilde)
        final_zeros = [z - T for z in zeros_combined]
        val = value_from_zeros(a_new, final_zeros)
        if val == target:
            c_new = compute_c_from_zeros(final_zeros)
            return a_new, final_zeros, c_new
        T += 1
    return None

In [15]:
S_list_dict = {
    1: [0],
    5: [0, 1],
    7: [0,2],
    19:[0,1,2]
}

def find_s_list_for_d(d, m_max, s_max):
    return S_list_dict[d]
    

In [16]:
def check_T_shift_inverse(X: int,
                          max_a: int = 20,
                          T_max: int = 2000,
                          m_max: int = 6,
                          s_max: int = 20) -> bool:
    """
    Check whether odd integer X can be reached by canonical T-shift
    from some predecessor N = (X - d)/4 for allowed d.

    - If X % 4 == 1: tries d = 1 and 5
    - If X % 4 == 3: tries d = 7 and 19
    Returns True if any valid T-shift construction from (a,b,c) of N yields X.
    """

    if X % 2 == 0:
        raise ValueError("X must be odd")

    # Determine d-values and corresponding S-lists based on mod_4 class
    if X % 4 == 1:
        d_candidates = [1, 5]
    else:
        d_candidates = [7, 19]

    # candidate predecessors
    candidates = []
    for d in d_candidates:
        if (X - d) % 4 == 0:
            N = (X - d) // 4
            if N > 0:
                candidates.append((N, d))

    if not candidates:
        return False

    # For each candidate N, enumerate possible lattice encodings (a, zeros)
    from itertools import combinations
    for N, d in candidates:
        for a in range(3, max_a + 1):
            for b in range(0, a + 1):
                # build all zero index combos of size b
                for zero_indices in combinations(range(a), b):
                    val = value_from_zeros(a, list(zero_indices))
                    if val == N:
                        # apply canonical T insertion
                        res = canonical_T_insertion(a, list(zero_indices),
                                                    d, T_max=T_max,
                                                    m_max=m_max, s_max=s_max)
                        if res is None:
                            continue
                        a_new, zeros_new, c_new = res
                        val2 = value_from_zeros(a_new, zeros_new)
                        if val2 == X:
                            print(f"Success: {N=} with d={d} -> {X}")
                            print(f"Predecessor bitstring length={a}, zeros={list(zero_indices)}")
                            print(f"Successor bitstring length={a_new}, zeros={zeros_new}")
                            return True
    return False

In [17]:
check_T_shift_inverse(37)

False

In [18]:
def collatzPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#


In [19]:
collatzPath(37)

'011100010110111'

In [20]:
TupChain(37)

[(0, 0, 0),
 (1, 1, 1),
 (2, 1, 1),
 (3, 1, 1),
 (4, 1, 1),
 (5, 2, 19),
 (6, 3, 89),
 (7, 4, 331),
 (8, 4, 331),
 (9, 5, 1249),
 (10, 5, 1249),
 (11, 5, 1249),
 (12, 6, 5795),
 (13, 6, 5795),
 (14, 6, 5795),
 (15, 6, 5795)]


(a, b, c) -> (a + 1, b + 1, c*3 + 2**a) -> (aa, bb, cc)
so:
(aa, bb, cc) <- (aa - 1, bb - 1, (cc - 2**(aa-1))//3)

In [72]:
def reverse_options(aa_bb_cc):
    aa, bb, cc = aa_bb_cc
    L = []
    L.append((aa-1, bb, cc))
    if bb > 0:
        xx = cc - (2**(aa-1))
        if (xx % 3) == 0:
            xx = xx//3
            L.append((aa-1, bb-1, xx))
    if len(L) == 0:
        raise ValueError(f"No legal paths from {aa_bb_cc}")
    return L
#

In [73]:
reverse_options((15, 6, 5795))

[(14, 6, 5795)]

In [74]:
def reverse_path_options(a_b_c):
    options = [[a_b_c]]
    for i in range(a_b_c[0]):
        for opt in list(options):
            aa_bb_cc = opt[-1]
            new_opts = reverse_options(aa_bb_cc)
            if len(new_opts) == 2:
                opt2 = [x for x in opt]
                opt2.append(new_opts[1])
                options.append(opt2)
            opt.append(new_opts[0])
    return options              

In [75]:
def reverse_path(a_b_c_list):
    """
    Call with a list of one a_b_c tuple of the item to search the path for
    returns the path found that ends in (0,0,0)
    """
    if a_b_c_list[-1][0] != 0:
        new_opts = reverse_options(a_b_c_list[-1])
        for new_opt in new_opts:
            result = reverse_path(a_b_c_list + [new_opt])
            if result != None:
                return result
    else:
        if a_b_c_list[-1] == (0,0,0):
            return a_b_c_list
        else:
            return None
#

In [76]:
RP = reverse_path([(15, 6, 5795)])
RP

[(15, 6, 5795),
 (14, 6, 5795),
 (13, 6, 5795),
 (12, 6, 5795),
 (11, 5, 1249),
 (10, 5, 1249),
 (9, 5, 1249),
 (8, 4, 331),
 (7, 4, 331),
 (6, 3, 89),
 (5, 2, 19),
 (4, 1, 1),
 (3, 1, 1),
 (2, 1, 1),
 (1, 1, 1),
 (0, 0, 0)]

In [45]:
def illegal_branches(RP):
    illegal_list = []
    for i in range(RP[0][0] - 1):
        opts = reverse_options(RP[i])
        for opt in opts:
            if opt != RP[i+1]:
                # print(f"{opt} != {RP[i+1]}")
                illegal_list.append([RP[i], opt])
    return illegal_list

In [46]:
illegal_branches(RP)

[[(14, 6, 5795), (13, 5, -799)],
 [(12, 6, 5795), (11, 6, 5795)],
 [(11, 5, 1249), (10, 4, 75)],
 [(9, 5, 1249), (8, 5, 1249)],
 [(7, 4, 331), (6, 4, 331)],
 [(6, 3, 89), (5, 3, 89)],
 [(5, 2, 19), (4, 2, 19)],
 [(3, 1, 1), (2, 0, -1)]]

# The right hand tuples above are not valid tuples in the lattice ... 
How can I recognize these instantly and cut the branch short?

For instance for generation 2, (2, 0, -1) is clearly illegal

Some rules:
- $c > 0$  ... right -- NOPE, c can be negative
- $c < 2^a$ -- NOPE
- what makes (10, 4, 75) illegal?  Not below lower bound ... 65, 73 are legal c-values for (10, 4, ?) ...

In [41]:
list(gen_generation(2))

[('11', (2, 0, 0), (4, 1)),
 ('10', (2, 1, 2), (2, 3)),
 ('01', (2, 1, 1), (1, 1)),
 ('00', (2, 2, 5), (-1, 9))]

In [52]:
# The legal c values for (a=10, b=4, c=?) 
sorted([tup[1][2] for tup in gen_generation(10) if tup[1][1] == 4])

[65,
 73,
 85,
 89,
 101,
 103,
 119,
 121,
 125,
 130,
 133,
 143,
 146,
 151,
 157,
 170,
 175,
 178,
 179,
 185,
 197,
 202,
 205,
 206,
 211,
 215,
 221,
 223,
 238,
 239,
 242,
 250,
 259,
 260,
 266,
 269,
 275,
 286,
 287,
 292,
 302,
 313,
 314,
 323,
 325,
 331,
 340,
 343,
 349,
 350,
 356,
 358,
 365,
 367,
 370,
 383,
 394,
 395,
 397,
 403,
 404,
 410,
 412,
 415,
 419,
 422,
 430,
 442,
 446,
 451,
 476,
 478,
 484,
 491,
 493,
 500,
 511,
 518,
 520,
 523,
 532,
 538,
 547,
 550,
 569,
 572,
 574,
 581,
 584,
 599,
 604,
 605,
 619,
 623,
 626,
 628,
 635,
 646,
 650,
 653,
 659,
 662,
 671,
 680,
 685,
 686,
 698,
 700,
 703,
 707,
 712,
 716,
 730,
 734,
 739,
 740,
 749,
 763,
 766,
 767,
 779,
 788,
 790,
 794,
 803,
 806,
 808,
 811,
 820,
 824,
 830,
 838,
 844,
 860,
 875,
 884,
 892,
 902,
 941,
 952,
 955,
 956,
 959,
 968,
 982,
 986,
 995,
 1000,
 1019,
 1022,
 1036,
 1040,
 1046,
 1064,
 1067,
 1076,
 1094,
 1100,
 1144,
 1148,
 1168,
 1208,
 1211,
 1238,
 12

In [55]:
[tup for tup in generate_S_vals(11) if tup[2] == 75]

[]

In [56]:
[tup for tup in generate_S_vals(11) if tup[2] == 65]

[(2, [0, 3, 5], 65, 1), (3, [0, 1, 2, 3], 65, 1)]